<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Наш план**
1. Загрузим и подготовим данные.
2. Обучим разные модели, с использованием кросс - валидации для большей уверенности в результате.
3. Сделаем общий вывод о проделанной разоте и сравним точность с константной моделью.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
# ячейка для импорта библиотек
import numpy as np
import pandas as pd 
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
import lightgbm as lgb
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords, wordnet 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## Подготовка

In [2]:
# скачиваем и открываем файл
try:
    comments = pd.read_csv(r'/datasets/toxic_comments.csv')
except:
    comments = pd.read_csv(r'https://code.s3.yandex.net/datasets/toxic_comments.csv')
    
display(comments.head(10))

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


Итак - у нас есть: странный столбец `Unnamed: 0` (точнее, кажется создано автосозданием номера строки в SQL или каким-то другим, но похожим способом) и два содержательных столбца - `text` (с комментариями на английском языке) и целевой столбец `toxic`, который означает является ли комментарий токсичным (негативным) или же нет.

In [3]:
# смотрим на общую информацию о наших данных
print(comments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None


Пропусков в данных нет, а вот комментариев много - более 159 тысяч.

In [4]:
# проверяем на явные дубликаты
print(comments.duplicated().sum())

0


Явных дубликкатов у нас нет. Но проверить всё равно следовало.

In [5]:
# изучаем баланс классов
print(comments.groupby('toxic').text.count())

toxic
0    143106
1     16186
Name: text, dtype: int64


У нас большой дисбаланс классов в выборке. Лишь немногим более 10% всех комментариев являются негативными, а все остальные - положительными (их чуть менее 90%). Для такого придётся ставить разный вес классам - иначе можем пропустить негативные комментарии. Это тем более важно, так как нам необходимо получать метрику F1 меру, для которой критически важны как точность, так и полнота.

In [6]:
# функции для очистки и лемматизации текста
def clear_text(text: str) -> str:
    cl_text = re.sub(r"[^a-zA-Z']", ' ', text)
    clear_lst = cl_text.split()
    cl_text = ' '.join(clear_lst).lower()
    return cl_text

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize(text: str) -> str:
    lemm_lst = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    lemm_text = ' '.join(lemm_lst)
    return lemm_text

In [8]:
nltk.download('averaged_perceptron_tagger')
# небольшая проверка работы функций
sentence1 = "The striped bats are hanging on their feet for best" 
sentence2 = "you should be ashamed of yourself went worked" 
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text']) 
print(df_my)

print(df_my['text'].apply(lambda x: lemmatize(clear_text(x))))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


                                                text
0  The striped bats are hanging on their feet for...
1      you should be ashamed of yourself went worked
0    the strip bat be hang on their foot for best
1       you should be ashamed of yourself go work
Name: text, dtype: object


Преобразование идёт успешно, можно применять на нашем датасете.

In [9]:
# проводим очистку и лемматизацию текста
tqdm.pandas()
comments['lemm_text'] = comments['text'].progress_apply(lambda x: lemmatize(clear_text(x)))

# проверяем лемматизацию
display(comments.head(10))

100%|██████████| 159292/159292 [19:35<00:00, 135.51it/s]


,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour i 'm see...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man i 'm really not try to edit war it 's ...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more i ca n't make any real suggestion on impr...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...
5,5,"""\n\nCongratulations from me as well, use the ...",0,congratulation from me a well use the tool wel...
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
7,7,Your vandalism to the Matt Shirvington article...,0,your vandalism to the matt shirvington article...
8,8,Sorry if the word 'nonsense' was offensive to ...,0,sorry if the word 'nonsense ' be offensive to ...
9,9,alignment on this subject and which are contra...,0,alignment on this subject and which be contrar...


Очистка и лемматизация проведены. Пора делить выборку на обучающую и тестовую части, составлять список стоп-слов дляя англиского языка.

In [10]:
# разделяем выборку на обучающую и тестовую части
features_train, features_test, target_train, target_test = train_test_split(comments['lemm_text'], comments['toxic'], \
                                                                            test_size=0.2, random_state=123, \
                                                                            stratify=comments['toxic'])
# сбрасываем индексы на всех выборках
features_train.reset_index(drop=True, inplace=True)
features_test.reset_index(drop=True, inplace=True)
target_train.reset_index(drop=True, inplace=True)
target_test.reset_index(drop=True, inplace=True)

In [18]:
# составляем список стоп - слов на английском
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))

# делаем из слов векторы для анализа
count_tf_idf = TfidfVectorizer(stop_words=stopwords, min_df=20)
count_tf_idf.fit(features_train.values)
tf_idf_train = count_tf_idf.transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)

# смотрим на размеры наших данных перед обучением моделей
print(tf_idf_train.shape)
print(tf_idf_test.shape)
print(target_train.shape)
print(target_test.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(127433, 9939)
(31859, 9939)
(127433,)
(31859,)


Итак: данные представлены в виде векторов. Едиственное, что нужно учитывать - даже при ограничении редких слов не менее 100 у нас получилось более 3500 столбцов - довольно много. В любом случае пора приступать к обучению моделей.

## Обучение

In [19]:
%%time

# обучаем гребневой классификатор
model_ridge = RidgeClassifier(class_weight='balanced', random_state=123)
gsearch_ridge = GridSearchCV(estimator=model_ridge, param_grid={'alpha': [0.1, 1.0, 10.0, 20.0, 50.0, 100.0]}, \
                             scoring='f1', cv=5)
gsearch_ridge.fit(tf_idf_train, target_train)
best_model_ridge = gsearch_ridge.best_estimator_
best_score_ridge = gsearch_ridge.best_score_
params_ridge = gsearch_ridge.best_params_
print(f'Метрика F1 гребневого классификатора при кросс-вaлидации равна: {round(best_score_ridge, 3)}')
print(f'Параметры гребневого классификатора: {params_ridge}')

Метрика F1 гребневого классификатора при кросс-вaлидации равна: 0.692
Параметры гребневого классификатора: {'alpha': 20.0}
CPU times: total: 41.1 s
Wall time: 21.6 s


Даже линейная модель при таких больших данных учится не быстро. Что и ожидалось. Качество относительно неплохое, но до требуемого не дотягивает (f1 = 0.665). Что будет о стальными моделями - время покажет.

In [20]:
%%time

# обучаем решающее дерево
model_tree = DecisionTreeClassifier(criterion='entropy', class_weight='balanced', random_state=123)
gsearch_tree = GridSearchCV(estimator=model_tree, scoring='f1', cv=5, \
                            param_grid={'max_depth': [5, 10, 15, 20, 25], 'min_samples_split': range(12, 28, 5)})
gsearch_tree.fit(tf_idf_train, target_train)
best_model_tree = gsearch_tree.best_estimator_
best_score_tree = gsearch_tree.best_score_
params_tree = gsearch_tree.best_params_
print(f'Метрика F1 решающего дерева при кросс-вaлидации равна: {round(best_score_tree, 3)}')
print(f'Параметры решающего дерева: {params_tree}')

Метрика F1 решающего дерева при кросс-вaлидации равна: 0.624
Параметры решающего дерева: {'max_depth': 25, 'min_samples_split': 12}
CPU times: total: 27min 10s
Wall time: 27min 15s


Оценка решающего дерева оказалась гораздо хуже гребневой регрессии (на локальном запуске, здесь намного лучше), можно предположить недоученность модели из-за большого числа столбцов с признаками.

In [21]:
%%time

# обучаем логистическую регрессию
model_logistic = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=123)
gsearch_logistic = GridSearchCV(estimator=model_logistic, scoring='f1', cv=5, \
                            param_grid={'penalty': ['l1', 'l2'], 'C': [0.1, 1.0, 5.0, 10.0, 15.0],\
                                        'max_iter': [400, 1000, 2000]})
gsearch_logistic.fit(tf_idf_train, target_train)
best_model_logistic = gsearch_logistic.best_estimator_
best_score_logistic = gsearch_logistic.best_score_
params_logistic = gsearch_logistic.best_params_
print(f'Метрика F1 логистической регрессии при кросс-вaлидации равна: {round(best_score_logistic, 3)}')
print(f'Параметры логистической регрессии: {params_logistic}')

Метрика F1 логистической регрессии при кросс-вaлидации равна: 0.731
Параметры логистической регрессии: {'C': 1.0, 'max_iter': 400, 'penalty': 'l1'}
CPU times: total: 6min 27s
Wall time: 4min 37s


Логистическая регрессия пока что показывает лучшие рещультаты по сравнению с моделями до неё, но и этого всё ещё не достаточно для достижения цели (F1 = 0.707). 

In [22]:
%%time

# обучаем градиентный бустинг
model_lgb = lgb.LGBMClassifier(verbosity=0, random_state=123)
gsearch_lgb = GridSearchCV(estimator=model_lgb, scoring='f1', cv=3, \
                            param_grid={'num_boost_round': range(150, 251, 50), 'learning_rate': np.arange(0.05, 0.16, 0.05),\
                                        'num_leaves': [31, 127]})
gsearch_lgb.fit(tf_idf_train, target_train)
best_model_lgb = gsearch_lgb.best_estimator_
best_score_lgb = gsearch_lgb.best_score_
params_lgb = gsearch_lgb.best_params_
print(f'Метрика F1 градиентного бустинга при кросс-вaлидации равна: {round(best_score_lgb, 3)}')
print(f'Параметры градиентного бустинга: {params_lgb}')

C:\anaconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=150, num_boost_round=150 will be ignored. Current value: num_iterations=150


C:\anaconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=150, num_boost_round=150 will be ignored. Current value: num_iterations=150


C:\anaconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[LightGBM] [Warning] num_iterations is set=250, num_boost_round=250 will be ignored. Current value: num_iterations=250
[LightGBM] [Warning] num_iterations is set=250, num_boost_round=250 will be ignored. Current value: num_iterations=250


C:\anaconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\anaconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=250, num_boost_round=250 will be ignored. Current value: num_iterations=250


C:\anaconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=200, num_boost_round=200 will be ignored. Current value: num_iterations=200
Метрика F1 градиентного бустинга при кросс-вaлидации равна: 0.768
Параметры градиентного бустинга: {'learning_rate': 0.1, 'num_boost_round': 200, 'num_leaves': 127}
CPU times: total: 2h 49min 3s
Wall time: 1h 25min 21s


По результатам обучения моделей лучше всего себя показала модель градиентного бустинга. Её и будем проверять на тестовой выборке.

In [23]:
# проверка градиентного бустинга на тесте
pred_lgb = best_model_lgb.predict(tf_idf_test)
f1_test_lgb = f1_score(target_test, pred_lgb)
print('F1 градиентного бустинга на тесте равна', f1_test_lgb)

F1 градиентного бустинга на тесте равна 0.7862687597032948


In [24]:
# сравниваем с константной моделью
model_dummy = DummyClassifier(strategy='uniform')
model_dummy.fit(tf_idf_train, target_train)
pred_dummy = model_dummy.predict(tf_idf_test)
f1_test_dummy = f1_score(target_test, pred_dummy)
print('F1 константной модели на тесте равна', f1_test_dummy)

F1 константной модели на тесте равна 0.16538199360285252


Наша модель показала результат по метрике f1 - 0.786 (с лемматизацией качество чуть хуже, но с округлением до 3 знаков после запятой - разницы нет), что и являлось нашей целью. И этот показатель в разы лучше константной модели, что показывает, что наша модель адекватна.

## Выводы

Мы смогли сделать для магазина инструмент, который будет искать токсичные комментарии и правильно сортировать их. По итогу этой моделью стал градиентный бустинг (LGBMClassifier) со следующими гиперпараметрами: {'learning_rate': 0.1, 'num_boost_round': 200, 'num_leaves': 127}.